In [17]:
from oracle import oracleTopd

data = oracleTopd('select * from recipe_data_table where rownum <= 100')


In [24]:
from tqdm import tqdm

In [30]:
from embedding import slicefood

toy = slicefood(data)


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\HwijunKwon\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py", line 3442, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\HwijunKwon\AppData\Local\Temp\ipykernel_8968\2149738528.py", line 3, in <module>
    toy = slicefood(data)
          ^^^^^^^^^^^^^^^
  File "c:\Users\HwijunKwon\inisw\recipe\models\embedding.py", line 34, in slicefood
NameError: name 'tqdm' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\HwijunKwon\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py", line 2057, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\HwijunKwon\AppData\Roaming\Python\Python311\site-packages\IPython\core\ultratb.py", line 1118, in structured_traceback
    return FormattedTB.s

In [26]:
from tqdm import tqdm
import ast
import re
toy = data[["recipe_title", "recipe_ingredients"]].copy()
# 문자열 전처리
toy["recipe_ingredients"] = toy["recipe_ingredients"].apply(lambda x: x.replace('\\ufeff', '').replace('\\u200b', ''))
# 새로운 칼럼 생성
for i in range(1, 21):
    toy.loc[:, f'ingredient{i}'] = None
    toy.loc[:, f'quantity{i}'] = None
    toy.loc[:, f'unit{i}'] = None
# 3. 식재료 칼럼 쪼개기
# 패턴과 일치하지 않는 데이터를 저장할 딕셔너리
non_matching_items = {}

for idx, row in tqdm(toy.iterrows(), total=toy.shape[0]): #tqdm으로 진행상황 확인
    ingredients_dict = ast.literal_eval(row["recipe_ingredients"])  # row["recipe_ingredients"] :{'재료': ['건새우50g', '물1/4컵', ...], '양념재료': ['설탕2큰술', '간장4큰술', ...}"
    ingredient_count = 1
    for category, items in ingredients_dict.items():
        if items:  # 아이템이 존재하는 경우
            for item in items:
                match = re.match(r'([가-힣]+(\([가-힣]+\))?)([\d.+/~-]*)([가-힣a-zA-Z]+|약간|조금)?', item)
                if match:
                    ingredient, _, quantity, unit = match.groups()
                    
                    toy.at[idx, f'ingredient{ingredient_count}'] = ingredient
                    toy.at[idx, f'quantity{ingredient_count}'] = quantity
                    toy.at[idx, f'unit{ingredient_count}'] = unit

                    ingredient_count += 1
                else:
                    # 패턴과 일치하지 않는 경우 딕셔너리에 추가
                    non_matching_items[idx] = item

# 패턴과 일치하지 않는 데이터 출력
for idx, item in non_matching_items.items():
    print(f'Row {idx}: {item}')


 20%|██        | 20/100 [00:00<00:00, 194.91it/s]

100%|██████████| 100/100 [00:00<00:00, 232.92it/s]

Row 24: 
Row 41: 
Row 60: 3cm두께커다란식빵1장


NameError: name 'data' is not defined

In [35]:
from embedding import convert_fraction_to_float, convert_unit_to_number
import pandas as pd

ingredient_columns = toy.filter(like='ingredient').drop(columns=['recipe_ingredients']) # 'ingredient'가 포함된 모든 열 선택
all_ingredients = [item for sublist in ingredient_columns.values for item in sublist if pd.notna(item)] # 각 행의 ingredient 값을 하나의 리스트로 결합


all_ingredients = set()
for i in range(1, 21):  
    all_ingredients.update(toy[f'ingredient{i}'].dropna().unique())

recipe_ingredients_df = pd.DataFrame(columns=list(all_ingredients))



recipe_rows = []
for idx, row in toy.iterrows():
    recipe_data = {ingredient: 0.0 for ingredient in all_ingredients}  # 모든 식재료를 None으로 초기화
    for i in range(1, 21):  
        ingredient = row[f'ingredient{i}']
        quantity = row[f'quantity{i}']
        unit = row[f'unit{i}']
        if pd.notna(ingredient) and pd.notna(quantity):
            quantity_float = convert_fraction_to_float(quantity)
            if quantity_float is not None:
                unit_number = convert_unit_to_number(unit) if pd.notna(unit) else 1
                recipe_data[ingredient] = quantity_float * unit_number
    recipe_rows.append(recipe_data)

# 새로운 데이터프레임 생성 (모든 식재료를 열로 가짐)
recipe_ingredients_df = pd.concat([pd.DataFrame([row]) for row in recipe_rows], ignore_index=True)

recipe_ingredients_df = recipe_ingredients_df.astype('float64')



In [39]:
recipe_ingredients_df.shape

(100, 392)

In [36]:
import numpy as np

a, b, c= np.linalg.svd(recipe_ingredients_df, full_matrices= True)